https://vgpena.github.io/classifying-tweets-with-keras-and-tensorflow/

En el anterior enlace, tenéis un ejemplo sobre cómo, a partir de tweets con un label específico (un sentimiento, positivo o negativo): 

1. Genera un conjunto de entrenamiento. El conjunto de entrenamiento es formado a partir de tweets completos pasados a un array con un tamaño específico.
2. Ese array (X_train de tamaño N) tiene un label que representa el sentimiento (y_train)
3. Como todas las frases tienen un tamaño N, la entrada de la red neuronal será de tamaño N y la salida de la red será de tamaño 2 usando activación softmax(porque hay dos clases).

Se pide: 

- Realizar un clasificador de reviews para el dataset de IMDB de la carpeta data_exercise/

**Cuando usa la importación "keras.x", reemplázalo por "tensorflow.keras.x"**

In [25]:
# importing libreries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow import keras
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

In [16]:
training = pd.read_csv("data_exercise/IMDB Dataset.csv")

In [17]:
X_train = training["review"]
y_train = training["sentiment"]

In [21]:
print(X_train.shape, y_train.shape)

(50000,) (50000,)


In [23]:
y_train = y_train.apply(lambda x: 1 if x == "positive" else 0)

In [27]:
# only work with the 3000 most popular words found in the dataset
max_words = 3000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

dictionary = tokenizer.word_index


def convert_text_to_index_array(text):
    # Make all texts the same length, the length of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in X_train:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# list of all tweets and sentiment converted to index arrays.
allWordIndices = np.asarray(allWordIndices)
y_train = np.array(y_train)

# create one-hot matrices out of the indexed tweets
X_train = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
y_train = keras.utils.to_categorical(y_train, 2)

In [28]:
# model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               1536512   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 1,668,354
Trainable params: 1,668,354
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [32]:
# training
model.fit(X_train, y_train,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Epoch 1/5
1407/1407 [==============================] - 10s 7ms/step - loss: 0.3367 - accuracy: 0.8549 - val_loss: 0.2894 - val_accuracy: 0.8782
Epoch 2/5
1407/1407 [==============================] - 9s 7ms/step - loss: 0.2459 - accuracy: 0.8973 - val_loss: 0.2731 - val_accuracy: 0.8800
Epoch 3/5
1407/1407 [==============================] - 9s 7ms/step - loss: 0.1895 - accuracy: 0.9227 - val_loss: 0.2861 - val_accuracy: 0.8842
Epoch 4/5
1407/1407 [==============================] - 9s 7ms/step - loss: 0.1260 - accuracy: 0.9494 - val_loss: 0.3661 - val_accuracy: 0.8822
Epoch 5/5
1407/1407 [==============================] - 9s 6ms/step - loss: 0.0780 - accuracy: 0.9701 - val_loss: 0.4085 - val_accuracy: 0.8770


In [36]:
# accuracy
model.evaluate(X_train, y_train)

1563/1563 [==============================] - 3s 2ms/step - loss: 0.0688 - accuracy: 0.9768


[0.06879564374685287, 0.9767799973487854]